In [1]:
import numpy as np
import pandas as pd
import os
import sys

In [2]:
df = pd.read_csv("../data/onlret_transaction.csv")

In [4]:
df.head(2)

,InvoiceNo,StockCode,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,Status,CustomerType
0,536365,85123A,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30,1,1
1,536365,71053,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34,1,1


In [5]:
df.columns

Index(['InvoiceNo', 'StockCode', 'Quantity', 'InvoiceDate', 'UnitPrice',
       'CustomerID', 'Country', 'TotalPrice', 'Status', 'CustomerType'],
      dtype='object')